## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-11-23-23-46-23 +0000,nyt,Israel Military Commanders Punished Over Hamas...,https://www.nytimes.com/2025/11/23/world/middl...
1,2025-11-23-23-38-00 +0000,wsj,BHP Abandons Bid for Anglo American Following ...,https://www.wsj.com/business/bhp-abandons-bid-...
2,2025-11-23-23-34-00 +0000,nypost,New X location tool exposes fake Gaza accounts...,https://nypost.com/2025/11/23/world-news/new-x...
3,2025-11-23-23-27-10 +0000,nypost,Zohran Mamdani eyes Eric Adams’ zoning czar fo...,https://nypost.com/2025/11/23/us-news/zohran-m...
4,2025-11-23-23-20-03 +0000,nyt,Top U.S. Military Officer to Visit Caribbean a...,https://www.nytimes.com/2025/11/23/us/politics...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2318/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
39,trump,29
47,ukraine,18
140,plan,16
44,peace,14
13,new,10


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
108,2025-11-23-14-38-13 +0000,nypost,Trump slams Ukraine’s lack of ‘gratitude’ in w...,https://nypost.com/2025/11/23/us-news/trump-sl...,114
19,2025-11-23-22-22-21 +0000,nyt,"Ukraine, U.S. Will Meet About Trump’s Peace Pl...",https://www.nytimes.com/2025/11/23/world/europ...,106
171,2025-11-23-03-22-00 +0000,wsj,Trump Says Ukraine Peace Plan Isn’t Final Afte...,https://www.wsj.com/politics/policy/trump-says...,92
29,2025-11-23-21-47-15 +0000,wapo,Ukraine seeks urgent changes to Trump peace pl...,https://www.washingtonpost.com/world/2025/11/2...,90
199,2025-11-23-00-00-00 +0000,wsj,Russia Stuck to Its Demands on Ukraine. Many A...,https://www.wsj.com/world/russia/russia-stuck-...,88


## Select top headlines

In [5]:
with open('exclude_sources.txt') as f:
    excluded_sources = {
        line.strip()
        for line in f
        if line.strip() and not line.strip().startswith('#')
    }

top_count = 10
top_rows = []
working = word_scores.copy()
remaining = latest.copy()

while len(top_rows) < top_count and not remaining.empty:
    scored_remaining = remaining.assign(
        score=remaining['title'].apply(
            lambda t: sum(
                working.get(w.lower(), 0)
                for w in re.findall(r'[A-Za-z]+', t)
                if len(w) > 1
            )
        )
    )
    ranked_loop = scored_remaining.sort_values('score', ascending=False)

    if ranked_loop.empty:
        break

    top_story = ranked_loop.iloc[0]
    remaining = remaining.drop(top_story.name)

    if top_story['source'] in excluded_sources:
        continue

    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for word in words:
        working.pop(word, None)

    top_rows.append(top_story[['score', 'pubdate', 'source', 'title', 'link']])

top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
108,114,2025-11-23-14-38-13 +0000,nypost,Trump slams Ukraine’s lack of ‘gratitude’ in w...,https://nypost.com/2025/11/23/us-news/trump-sl...
61,41,2025-11-23-19-31-39 +0000,bbc,Israel kills top Hezbollah official in first a...,https://www.bbc.com/news/articles/cn81j54xjx1o...
125,40,2025-11-23-12-00-00 +0000,wsj,While President Trump and New York Mayor-elect...,https://www.wsj.com/economy/everyone-is-talkin...
114,32,2025-11-23-13-39-57 +0000,nypost,"Ukraine peace plan was authored by US, Rubio s...",https://nypost.com/2025/11/23/world-news/ukrai...
32,31,2025-11-23-21-41-00 +0000,nypost,Glam former prime minister Sanna Marin complai...,https://nypost.com/2025/11/23/world-news/glam-...
183,27,2025-11-23-02-00-00 +0000,wsj,How CEOs got more comfortable with tariffs: We...,https://www.wsj.com/economy/trade/ceo-tariffs-...
18,24,2025-11-23-22-28-38 +0000,nypost,Married ex-NFL player who now heads athletics ...,https://nypost.com/2025/11/23/us-news/ex-vikin...
74,24,2025-11-23-18-07-23 +0000,nypost,"NYC house fire kills 95-year-old, critically i...",https://nypost.com/2025/11/23/us-news/nyc-hous...
41,22,2025-11-23-20-48-21 +0000,nypost,"Long Island’s largest zoo, which faced animal ...",https://nypost.com/2025/11/23/us-news/long-isl...
162,21,2025-11-23-05-23-55 +0000,nypost,Canadian cop admits targeting dozens of women ...,https://nypost.com/2025/11/23/world-news/cop-a...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
